## Molecular dynamics simulation of the paper of Kim 
Skjegstad, L. E. J., Nickels, J. F., Sneppen, K. & Kirkegaard, J. B. Epigenetic switching with asymmetric bridging interactions. Biophysical Journal 122, 2421–2429 (2023).


In [48]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy.special import lambertw

"The dynamics of the chromatin is described by an over-damped Langevin equation of the form: 
$$ d\textbf{X}_i(t) = - \nabla_{X_i} U (\textbf{X}(t)) dt + d\textbf{W}_i(t) $$
where $\textbf{X}_i$ is the position of the i’th monomer, U is the aggregate potential function, and $\textbf{W}$ is a Wiener process satisfying $<\textbf{W}_i(t) , \textbf{W}_i(t')> = 3\sigma^2\delta(t-t')$, with $\sigma$ setting the diffusive noise scale."

"The potential U depends on the positions and states of all other monomers : 
$$ U \propto [\sum_{ij} e^{-4r_{ij}/{l_0}} - \sum_{ij  \in  bonds} e^{-4br_{ij}/{l_0}}]$$

Here, the terms in the first summation account for repulsion between all monomers, and the terms in the second summation account for attraction for the S-state monomers that form bonds. For bonded monomers, the two exponential terms create a potential well with an equilibrium distance equal to $l_0/2$, ensured by setting $b = - LambertW(-2e^{-2})/2$. We note that U, as specified here, does not constitute the total potential, as, again, adjacent monomers in the polymer are constrained to be distance $l_0$ from one another."

As in the article we will use the Euler-Marayuma algorithm to resolve the 1st equation

In [49]:
def compute_b_lambertw(l0):
    b_solution = lambertw(-2 * np.exp(-2)) / 2
    return b_solution.real

In [50]:
def potential_energy(positions, bonds, l0, b):
    repulsion_term = np.sum(np.exp(-4 * np.abs(positions[:, np.newaxis] - positions) / l0))
    attraction_term = np.sum(np.exp(-4 * b * np.abs(positions[:, np.newaxis] - positions[bonds]) / l0))
    return repulsion_term - attraction_term

In [122]:
def potential_gradient(positions, bonds, l0, b):
    repulsion_gradient = -4 * np.sum(np.sign(positions[:, np.newaxis] - positions) *
                                     np.exp(-4 * np.abs(positions[:, np.newaxis] - positions) / l0) / l0, axis=1)

    attraction_gradient = 4 * b * np.sum(np.sign(positions[:, np.newaxis, np.newaxis] - positions[bonds]) *
                                          np.exp(-4 * b * np.abs(positions[:, np.newaxis, np.newaxis] - positions[bonds]) / l0) / l0, axis=1)
    
    print(repulsion_gradient)
    print(attraction_gradient)

    return repulsion_gradient - attraction_gradient

In [123]:
def langevin_dynamics(positions, bonds, l0, b, diffusion_coefficient, time_step, num_steps):
    num_particles = len(positions)
    trajectory = [positions.copy()]

    for _ in range(num_steps):
        # Compute the noise term
        noise = np.random.normal(0, np.sqrt(6 * diffusion_coefficient * time_step), num_particles)

        # Compute the potential gradient
        grad_potential = potential_gradient(positions, bonds, l0, b)

        # Update positions using overdamped Langevin dynamics
        positions = positions - grad_potential * time_step + noise

        # Store the updated positions in the trajectory
        trajectory.append(positions.copy())

    return np.array(trajectory)

In [124]:
# Parameters
num_particles = 20
diffusion_coefficient = 1.0
time_step = 0.01
num_steps = 1000
l0 = 1.0  # equilibrium distance
b = compute_b_lambertw(l0)

In [125]:
# Example bonded pairs 
bonds = np.array([[i, (i + 1) % num_particles] for i in range(num_particles - 1)])

In [126]:
# Initial random positions 
initial_positions = np.random.randn(num_particles)

In [127]:
# Run Langevin dynamics simulation
trajectory = langevin_dynamics(initial_positions, bonds, l0, b, diffusion_coefficient, time_step, num_steps)

ValueError: operands could not be broadcast together with shapes (20,1) (19,2) 

In [ ]:
# Plot the trajectory
for i in range(num_particles):
    plt.plot(trajectory[:, i], label=f'Particle {i + 1}')

NameError: name 'trajectory' is not defined